# LLM - RAG (Large Language Models - Retrievel-Augmented Generation)

# Overview
This demo showcases how to use the GPT-4 model in a Retrieval-Augmented Generation (RAG) setup. \
The model analyzes a natural language query, retrieves relevant information from a knowledge base, and provides an answer based on the retrieved information.



In [1]:
import re
import requests
import sys
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2024-02-15-preview"
)

CHAT_COMPLETIONS_MODEL = os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME')

### Start with a natural question

In [2]:
input = "Which presendential candidate is likely to win in 2024 election? Explain the reasons."

### Step 1: GPT-4o: What do I need to to answer this question?

In [6]:
system_prompt=f'''Answer the following questions as best you can. You have access to the following tools:

Web Search: Use the web to find information

You need to follow STRICTLY below format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Research]
Action Input: the input to the action

Begin!
'''
user_prompt = f'''
Question: {input}
Thought:'''

In [7]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":system_prompt},
                {"role":"user","content": user_prompt,}],
        max_tokens=400,)

print(response.choices[0].message.content)

Thought: To determine which presidential candidate is likely to win the 2024 election, it is important to consider recent polling data, political analysis, and expert opinions. I will perform a web search to gather the most up-to-date information regarding the candidates and their standings. 

Action: Research
Action Input: 2024 US presidential election candidates leading poll numbers and analysis


### Step 2: Search web for more details 
Before continuing please create a Bing Search Service in your subscription [Create Bing Search Resource](https://learn.microsoft.com/en-us/bing/search-apis/bing-web-search/create-bing-search-service-resource). \
This sample makes a call to the Bing Web Search API with a query and returns relevant web search.

In [8]:
# Use Azure BING Search API to get the results
import requests
import json
url = "https://api.bing.microsoft.com/v7.0/search"
headers = {"Ocp-Apim-Subscription-Key": os.getenv('BING_SUBSCRIPTION_KEY')}
params = {"q": "2024 presidential election candidates and current political analysis"}
response = requests.get(url, headers=headers, params=params)

In [9]:
context = ''
print(response.json())
for result in response.json()['webPages']['value']:
    context += result['snippet'] + '\n'

{'_type': 'SearchResponse', 'queryContext': {'originalQuery': '2024 presidential election candidates and current political analysis'}, 'webPages': {'webSearchUrl': 'https://www.bing.com/search?q=2024+presidential+election+candidates+and+current+political+analysis', 'totalEstimatedMatches': 476000, 'value': [{'id': 'https://api.bing.microsoft.com/api/v7/#WebPages.0', 'name': 'Analysis: Almost everyone thinks the 2024 election results are ...', 'url': 'https://www.cnn.com/2024/11/20/politics/2024-results-legitimate-analysis/index.html', 'datePublished': '2024-11-20T11:00:00.0000000', 'datePublishedDisplayText': '20 בנוב׳ 2024', 'isFamilyFriendly': True, 'displayUrl': 'https://www.cnn.com/2024/11/20/politics/2024-results-legitimate-analysis', 'snippet': 'Overall, 18% of adults said Trump’s win wasn’t legitimate, three times as large as the figures for the 2024 election By April 2017, the percentage of adults who viewed Trump as an illegitimate ...', 'deepLinks': [{'name': 'Election 2024: 

### Collected information for further processing

In [10]:
print(context)

Overall, 18% of adults said Trump’s win wasn’t legitimate, three times as large as the figures for the 2024 election By April 2017, the percentage of adults who viewed Trump as an illegitimate ...
Four years after leaving Washington as a pariah, following his attempt to overturn the 2020 election to stay in office, Trump’s victory defied two assassination attempts, two presidential ...
Follow the AP’s coverage of the 2024 presidential elections, including early voting, election results, Electoral College maps, and more.
President Biden and former President Donald J. Trump cruised to victory in their respective parties’ primaries. Mr. Trump dispatched with a large group of Republican challengers, most of whom...
2024 election guide: Presidential candidates, polls, primaries and caucuses, voter information and results for November 5, 2024
Donald Trump has won the U.S. presidential election, defeating Kamala Harris. Here’s the latest on what you need to know.
It’s Election Day, and the po

In [11]:
system_prompt=f'''Answer the following questions as best you can. Use the context provided below:
{context}

Begin!
'''
user_prompt = f'''
Question: {input}\
Answer:'''

### Step 3: GPT-4o: What do I need to do to answer this question? (Again)

In [12]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":system_prompt},
                {"role":"user","content": user_prompt,}],
        max_tokens=400,)

print(response.choices[0].message.content)

Based on the context provided, Donald Trump has won the 2024 U.S. presidential election, defeating Kamala Harris. The information suggests that Trump succeeded despite facing a competitive race, as indicated by polls showing it to be one of the closest presidential races in American history. Additionally, Trump overcame significant challenges within his own party during the primaries, dispatching a large group of Republican challengers.

Reasons for Trump's victory could include his strong support among Republican base voters, effective campaign strategy, and ability to rally voters despite facing two assassination attempts and previous controversies around his attempts to overturn the 2020 election. Trump's victory indicates that he was able to resonate with enough voters nationwide, or particularly in key battleground states, to secure a win in the Electoral College. However, this analysis is speculative in nature due to the lack of detailed polling data within the provided context.


### GPT-4o: I now have final answer to the question. 

#### Use cases to explore
1. **Knowledge Management** \
Improve access to corporate knowledge by retrieving and presenting information from internal documents, manuals, and policies.

2. **Business Intelligence and Analytics** \
Support market analysts by retrieving relevant market data, reports, and insights from various sources.

3. **Legal and Compliance** \
Aid legal professionals by retrieving relevant case laws, statutes, and legal precedents based on queries.